In [1]:
from typing import Tuple, List, Dict, Any
import polars as pl
import pandas as pd
from pprint import pprint, pformat
import sys
from pathlib import Path
from tqdm import tqdm, trange
import psutil
import re


# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.utils import process_lazyframe_in_chunks
from code.loading import DataLoader
from code.preprocess import TextPreprocessor, create_udi_preprocessor, create_company_preprocessor
from code.preprocess.preprocess import get_pattern_cols, \
    get_unique_by_cols_safe, get_unique, \
    analyze_null_values, replace_pattern_with_null

cleaned_path = DATA_DIR / 'silver' / 'maude_cleaned.parquet'

In [2]:
# maude 데이터 불러오기
loader1 = DataLoader(
    start=2020,
    end=2025,
    output_file = DATA_DIR / 'maude.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
maude_lf = loader1.load(adapter=adapter, **polars_kwargs)
maude_lf


📖 /home/dataiku/eric/Sparta/Project3/data/maude.parquet 로딩 중... (adapter=polars)


In [3]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=DATA_DIR/'udi.parquet',
)

udi_lf = udi_loader.load(adapter, **polars_kwargs)
udi_lf


📖 /home/dataiku/eric/Sparta/Project3/data/udi.parquet 로딩 중... (adapter=polars)


## UDI Dataset 전처리

### Primary 추가

In [4]:
IDENTIFIER_PATTERNS = [
    r"^device_\d+_brand_name$",
    r"identifiers_\d+_id", 
    r"identifiers_\d+_issuing_agency", 
    r"identifiers_\d+_package_discontinue_date", 
    r"identifiers_\d+_package_status", 
    r"identifiers_\d+_package_type", 
    r"identifiers_\d+_quantity_per_package", 
    r"identifiers_\d+_type", 
    r"identifiers_\d+_unit_of_use_id"
]
UDI_DI_PATTERNS = [r'^identifiers_\d+_id$']
TYPE_PATTERNS = [
    r'identifiers_\d+_type'
]

CUSTOMER_PATTERNS = [r'^customer']
DEVICE_SIZE_PATTERNS = [r'^device_sizes']
STORAGE_PATTERNS = [r'^storage']

## Drop 필요없는 열

In [5]:
# 패턴 합치기
drop_patterns = CUSTOMER_PATTERNS + DEVICE_SIZE_PATTERNS + STORAGE_PATTERNS

# LazyFrame에서 패턴에 맞는 컬럼 제외
udi_lf = udi_lf.select(pl.exclude(f"^({'|'.join(p.strip('^') for p in drop_patterns)})"))

In [6]:

udi_di_cols = get_pattern_cols(udi_lf, UDI_DI_PATTERNS)
identifiers_cols = get_pattern_cols(udi_lf, IDENTIFIER_PATTERNS)
type_cols = get_pattern_cols(udi_lf, TYPE_PATTERNS)

In [7]:
# type-udi_di 쌍 만들기 (인덱스로 매칭)
def extract_index(col_name):
    match = re.search(r'identifiers_(\d+)_', col_name)
    return int(match.group(1)) if match else None

type_id_pairs = []
for type_col in type_cols:
    idx = extract_index(type_col)
    udi_di_col = f'identifiers_{idx}_id'
    if udi_di_col in udi_di_cols:
        type_id_pairs.append((type_col, udi_di_col))

len(type_id_pairs)

132

In [8]:
primary_udi_unique = set()

for type_col, id_col in tqdm(type_id_pairs, desc="Processing columns", unit="col"):
    try:
        count = (
            udi_lf
            .filter(pl.col(type_col).eq("Primary"))
            .select(pl.len())
            .collect()
            .item()
        )
        
        if count > 0:
            ids = (
                udi_lf
                .filter(pl.col(type_col).eq("Primary"))
                .select(pl.col(id_col))
                .unique()
                .collect()
                .to_series()
                .drop_nulls()
                .to_list()
            )
            primary_udi_unique.update(ids)
            
    except Exception as e:
        tqdm.write(f"Error processing {type_col}: {e}")
        continue

print(f"\n{'='*50}")
print(f"UDI 데이터의 고유 Primary udi 개수: {len(primary_udi_unique):,}")
print(f"{'='*50}")

Processing columns: 100%|██████████| 132/132 [00:01<00:00, 75.44col/s] 


UDI 데이터의 고유 Primary udi 개수: 4,903,764


In [9]:
step1_path = DATA_DIR / 'silver' / 'udi_primary.parquet'
step2_path = DATA_DIR / 'silver' / 'udi_clean.parquet'

In [10]:
# Primary인 id를 추출
def primary_transform(lf: pl.LazyFrame):
    return lf.with_columns(
        pl.coalesce([
            pl.when(pl.col(type_col).eq("Primary"))
            .then(pl.col(id_col))
            for type_col, id_col in type_id_pairs
        ]).alias('primary_udi_di')
    )

process_lazyframe_in_chunks(
    udi_lf, 
    primary_transform, 
    step1_path, 
    10_000,
    desc="Primary extraction"
)

# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=step1_path,
)

primary_udi_lf = udi_loader.load(adapter, **polars_kwargs)

Processing 4,903,764 rows in chunks of 10,000...


Primary extraction: 100%|██████████| 491/491 [01:36<00:00,  5.07it/s]


Merging chunks...
✓ Saved to /home/dataiku/eric/Sparta/Project3/data/silver/udi_primary.parquet

📖 /home/dataiku/eric/Sparta/Project3/data/silver/udi_primary.parquet 로딩 중... (adapter=polars)


In [11]:
# 회사 이름 정제
create_company_preprocessor().apply_to_lazyframe(
    primary_udi_lf,
    'company_name',
    step2_path,
    10_000,
)

step1_path.unlink(missing_ok=True) 

# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=step2_path,
)

cleaned_udi_lf = udi_loader.load(adapter, **polars_kwargs)

[CompanyName] Extracting unique values from 1 column(s)...
[CompanyName] Creating mapping for 11,258 unique values...
  - Kept: 11,257 (100.0%)
  - Deleted: 1 (0.0%)
Processing 4,903,764 rows in chunks of 10,000...


[CompanyName] Processing 1 column(s): 100%|██████████| 491/491 [01:37<00:00,  5.02it/s]


Merging chunks...
✓ Saved to /home/dataiku/eric/Sparta/Project3/data/silver/udi_clean.parquet

📖 /home/dataiku/eric/Sparta/Project3/data/silver/udi_clean.parquet 로딩 중... (adapter=polars)


In [16]:
udi_lf.select(['identifiers_0_id', 'company_name']).filter(
    pl.col('identifiers_0_id').eq('06972946643877')
).head(10).collect().to_pandas().transpose()

,0
identifiers_0_id,06972946643877
company_name,None


In [ ]:
cleaned_udi_lf.filter(
    pl.col('primary_udi_di').eq('00887761000872')
).head(10).collect().to_pandas().transpose()

## UDI 클린징

In [ ]:
step1_path = DATA_DIR / 'silver' / 'clean_step1.parquet'
step2_path = DATA_DIR / 'silver' / 'clean_step2.parquet'

# 1단계: UDI 클린징
create_udi_preprocessor().apply_to_lazyframe(
    maude_lf, 'device_0_udi_di', step1_path, chunk_size=10_000
)

In [ ]:

# 2단계: 회사명 클린징
maude_lf2 = pl.scan_parquet(step1_path)
create_company_preprocessor().apply_to_lazyframe(
    maude_lf2, 'device_0_manufacturer_d_name', step2_path, chunk_size=10_000
)

In [ ]:
# maude 데이터 불러오기
loader3 = DataLoader(
    start=2020,
    end=2025,
    output_file = step2_path,
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
maude_lf = loader3.load(adapter=adapter, **polars_kwargs)
maude_lf

## 고유값 추출

In [ ]:
cols_group = {
    'udi': udi_di_cols,
}

# udi 데이터셋의 udi_di 고유값
udi_udi_unique = get_unique_by_cols_safe(
    udi_lf, 
    cols_group,
    memory_safety_ratio=0.3,
    calibration_factor = 1
)['udi']

In [ ]:
maude_udi_unique = get_unique(maude_lf, ['device_0_udi_di'])
angry_udi_unique = maude_udi_unique - udi_udi_unique
survive_udi_unique = maude_udi_unique & udi_udi_unique

print(f'UDI 데이터의 고유 udi 개수: {len(udi_udi_unique)}개')
print(f'MAUDE 데이터의 고유 udi 개수: {len(maude_udi_unique)}개')
print(f'UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: {len(angry_udi_unique)}개')
print(f'UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: {len(survive_udi_unique)}개')

In [ ]:
maude_primary_udi_unique = survive_udi_unique & primary_udi_unique
maude_secondary_udi_unique = survive_udi_unique - primary_udi_unique

print(f'Primary UDI인 MAUDE 데이터의 고유 udi 개수: {len(maude_primary_udi_unique)}개')
print(f'Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: {len(maude_secondary_udi_unique)}개')

In [ ]:
class UniqueUDIDI:
    def __init__(
        self,
        udi_udi_unique: set = None,
        maude_udi_unique: set = None,
        angry_udi_unique: set = None,
        survive_udi_unique: set = None,
        primary_udi_unique: set = None,
        maude_primary_udi_unique: set = None,
        maude_secondary_udi_unique: set = None
    ):
        self.udi = udi_udi_unique
        self.maude = maude_udi_unique
        self.angry = angry_udi_unique
        self.survive = survive_udi_unique
        self.primary = primary_udi_unique
        self.maude_primary = maude_primary_udi_unique
        self.maude_secondary = maude_secondary_udi_unique

    def print_stats(self):
        print(f'UDI 데이터의 고유 udi 개수: {len(self.udi)}개')
        print(f'UDI 데이터의 고유 Primary udi 개수: {len(self.primary)}개')
        print(f'MAUDE 데이터의 고유 udi 개수: {len(self.maude)}개')
        print(f'UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: {len(self.survive)}개')
        print(f'Primary UDI인 MAUDE 데이터의 고유 udi 개수: {len(self.maude_primary)}개')
        print(f'Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: {len(self.maude_secondary)}개')
        print(f'UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: {len(self.angry)}개')

unique_udi_di = UniqueUDIDI(udi_udi_unique, maude_udi_unique, angry_udi_unique, survive_udi_unique, primary_udi_unique, maude_primary_udi_unique, maude_secondary_udi_unique)

unique_udi_di.print_stats()

## UDI 데이터셋에 Primary 컬럼 생성

In [ ]:
# MAUDE UDI를 LazyFrame으로
maude_udi_lf = pl.LazyFrame({
    'udi_di': list(unique_udi_di.maude_secondary)
})

In [ ]:
necessary_cols = [
    'company_name',
    'brand_name',
    'version_or_model_number',
    'catalog_number',
    'primary_udi_di',
]

# unpivot 후 즉시 필터링 (메모리 증가 최소화)
udi_mapping_lf = (
    udi_with_primary_lf
    .with_row_index('row_idx')
    .select(['row_idx'] + necessary_cols + udi_di_cols)
    .unpivot(
        index=['row_idx'] + necessary_cols,
        on=udi_di_cols,
        variable_name='matched_col',
        value_name='udi_di'
    )
    .filter(
        pl.col('udi_di').is_not_null() &  # null 제거
        pl.col('udi_di').is_in(unique_udi_di.maude_secondary)  # 매칭되는 것만
    )
    .unique(subset=['udi_di', 'row_idx'])
    .select(['udi_di', 'row_idx'] + necessary_cols)
)

In [ ]:
udi_mapping_df = udi_mapping_lf.collect().to_pandas()

In [ ]:
# 통계만 먼저 확인
print(f"Total matches: {len(udi_mapping_df):,}")
print(f"Unique UDIs: {udi_mapping_df['udi_di'].nunique():,}")

In [ ]:
udi_mapping_df[udi_mapping_df['udi_di'] == '00021292007706']

In [ ]:
maude_lf.filter(
    pl.col('device_0_udi_di').eq('00021292007706')
).head(15).collect().to_pandas().transpose()

# 전처리 함수 설계

In [ ]:
# 일치 점수 매기는 함수
# 1. 매핑한 행만 불러오기
# 2. company_name, brand_name, version_or_mode_number, catalog_number에 대하여 
# 2-1. 전부 대문자로
# 2-2. 일치 점수 매기기
# 2-3. 가장 점수가 높은 것의 primary_udi_di를 할당
# 2-4. 필요한 정보 가져오기


# udi를 깔끔하게 만듦
# primary: 그대로 primary_lf와 매칭해서 필요 info 가져옴
# secondary: primary_lf 중 udi 매칭한 lf에서 점수 높은 info 가져옴
# null: primary_lf에서 점수 높은 info 가져옴

## MAUDE 데이터의 UDI를 깔끔하게

In [ ]:
with open('primary.txt', 'w', encoding='utf-8') as f:
    pretty_primary = pformat(unique_udi_di.maude_primary, indent=4, width=80)
    f.write(pretty_primary)
    

with open('secondary.txt', 'w', encoding='utf-8') as f:
    pretty_secondary = pformat(unique_udi_di.maude_secondary, indent=4, width=80)
    f.write(pretty_secondary)

with open('notfound.txt', 'w', encoding='utf-8') as f:
    pretty_angry = pformat(unique_udi_di.angry, indent=4, width=80)
    f.write(pretty_angry)

In [ ]:
group_cols = ['device_0_manufacturer_d_name', 'device_0_brand_name','device_0_model_number', 'device_0_lot_number']
na_pattern = r'^UNK|^(NI|NA|N/A|ASKU|NAV|NAVU|MSK|QS|TRC|DER|INV|PINF|NINF|-)$'
maude_lf = replace_pattern_with_null(maude_lf, group_cols + ['device_0_udi_di'], na_pattern)

In [ ]:
group_lf = maude_lf.select(['device_0_udi_di', *group_cols]).group_by(group_cols).agg(
    pl.col('device_0_udi_di').n_unique().alias('udi_di_unique'),
)

outlier = group_lf.filter(
    pl.col('udi_di_unique').gt(1)
).select(pl.len()).collect().item()

print(f'UDI-DI 이상치는 {outlier}개 있습니다.')
group_lf.collect().drop_nulls().sort('udi_di_unique', descending=True).head(100).to_pandas()

In [ ]:
maude_lf.select(group_cols + ['device_0_udi_di']).filter(
    pl.col('device_0_manufacturer_d_name').eq('DEXCOM, INC.'),
    pl.col('device_0_model_number').eq('9500-161'),
    pl.col('device_0_brand_name').eq('DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM'),
).drop_nulls().head(1000).collect().to_pandas()